In [1]:
from tokenizers.implementations import ByteLevelBPETokenizer
import re
import unidecode
import numpy as np
from pprint import pprint
from keras.models import load_model
import json

Using TensorFlow backend.
/media/Datos/Documentos/Python/anacondas/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
model=load_model('results/tokenizer/model_weights_best.h5')


W0419 20:52:41.772816 139909460760384 deprecation_wrapper.py:119] From /media/Datos/Documentos/Python/anacondas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:58: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0419 20:52:41.775372 139909460760384 deprecation_wrapper.py:119] From /media/Datos/Documentos/Python/anacondas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:442: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0419 20:52:41.778656 139909460760384 deprecation_wrapper.py:119] From /media/Datos/Documentos/Python/anacondas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3543: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0419 20:52:41.851371 139909460760384 deprecation_wrapper.py:119] From /media/Datos/Documentos/Python/anacondas/anaconda3/lib/python3.7/site-packages/keras/backend/

In [3]:
def clean_str(s):
    s = unidecode.unidecode(s)
    s = re.sub(r'\t','',s)
    s = re.sub(r'\r','',s)
    s = s.lower()
    s = re.sub(r'[^a-z0-9]',' ',s)
    s = re.sub(r' +',' ',s)
    return s.strip()


In [4]:
tokenizer = ByteLevelBPETokenizer(
    "results/tokenizer/spoilers-vocab.json",
    "results/tokenizer/spoilers-merges.txt",
)

In [5]:
clf_token = tokenizer.token_to_id('[CLS]')  # classifier token
pad_token = tokenizer.token_to_id('[PAD]')  # pad token
max_length = 200

def token2id(text):
    "Convert text (item[0]) to sequence of IDs and label (item[1]) to integer"
    inputs = tokenizer.encode(text).ids
    # Trim or pad dataset
    if len(inputs) >= max_length:
        inputs = inputs[:max_length - 1]
        ids = inputs + [clf_token]
    else:
        pad = [pad_token] * (max_length - len(inputs) - 1)
        ids = inputs + [clf_token] + pad
    return ids

def text2tensor(texts):
    out=[]
    for text in texts:
        out.append(token2id(text))
    out = np.array(out)#.transpose(1,0)
    return out

In [6]:
def spoilme(s):
    s = re.sub(r'\n','.',s)
    s = s.split('.')
    s = [clean_str(i) for i in s]
    s = [f for f in s if f]
    s_ = text2tensor(s)
    predictions = model.predict(s_)
    predictions_ = predictions.argmax(axis=1)
    result = list(zip(s, predictions_))
    spoilers = [r[0] for r in result if r[1]==1]
    print(len(spoilers)/len(result))
    return spoilers , predictions

In [21]:
s='''
Sinopsis
Una historia multigeneracional de una familia taiwanesa que abarca desde 1950 hasta la actualidad. (FILMAFFINITY)
Críticas
"Un drama de inmigrantes emotivo y magnífico (...) Yang imbuye a su historia, honesta y semi-ficcional, con las evocadoras tradiciones narrativas del cine asiático moderno, desde Wong Kar Wai hasta Edward Yang" 
Eric Kohn: IndieWire 
"Soporífera, caótica y poco reflexiva (...) Parece resuelta a no desarrollar ninguno de sus personajes secundarios" 
Inkoo Kang: The Hollywood Reporter 
"Romántica, reflexiva y desgarradora, todo a la vez (...) Un excelente debut de Yang" 
Sandy Schaefer: Screen Rant 
"En lo que se destaca de otros dramas de su estilo, es en su conmovedora exploración de las consecuencias emocionales, pocas veces vistas, de los que persiguen decididamente el Sueño Americano" 
Katie Rife: AV Club 
"Aunque imperfecta, describe un aspecto de la experiencia [inmigratoria] de primera generación en América, que rara vez se ha captado en pantalla" 
Peter Debruge: Variety 
"Hay una verdadera tensión emocional en la manera en la que Yang conjura el pasado (...) Una película seria y bien interpretada sobre el sacrificio y el arrepentimiento (...) Puntuación: ★★★ (sobre 5)" 
Peter Bradshaw: The Guardian 
"Es fácil entender que la decepción de su protagonista podría desgastarle, pero la frialdad con la que trata a la mujer resulta abrupta e injustificada. La obra es especialmente tosca cuando aborda escenas en la actualidad." 
Kristen Yoonsoo Kim: The New York Times 
"Una película consciente, emotiva y en algunos momentos realmente brillante que demuestra el potencial de Yang en el mundo del cine." 
Lo único un poco malo es cuando Darth Vader le dice a Luke que es su padre. Leia muered haci la mitad7


'''

In [22]:
spoilme(s)

0.07407407407407407


(['lo unico un poco malo es cuando darth vader le dice a luke que es su padre',
  'leia muered haci la mitad7'],
 array([[0.9368215 , 0.06403267],
        [0.5793802 , 0.42037097],
        [0.8974633 , 0.10385913],
        [0.9163651 , 0.08476657],
        [0.9051142 , 0.09559482],
        [0.8914978 , 0.10788915],
        [0.9172055 , 0.08380356],
        [0.9769381 , 0.02328405],
        [0.8296634 , 0.17111143],
        [0.78581494, 0.2166293 ],
        [0.8587893 , 0.14198062],
        [0.93829453, 0.06232443],
        [0.9290909 , 0.07164249],
        [0.8019627 , 0.19442189],
        [0.7328732 , 0.2697453 ],
        [0.74993646, 0.2475591 ],
        [0.8870927 , 0.11401105],
        [0.601317  , 0.39838186],
        [0.7857888 , 0.21435952],
        [0.9499253 , 0.05067292],
        [0.63848186, 0.36454695],
        [0.5397742 , 0.45699012],
        [0.8482833 , 0.15229267],
        [0.55384046, 0.44857398],
        [0.97308975, 0.02662445],
        [0.05387169, 0.9461588 ],
   